In [1]:
# this cell for running in Google Colab
!pip install -Uqq fastbook fastai

     |████████████████████████████████| 727kB 9.3MB/s 
     |████████████████████████████████| 204kB 16.9MB/s 
     |████████████████████████████████| 1.2MB 17.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 


In [2]:
import fastbook

In [3]:
# this cell will setup a path that allows you to load and save files to your google drive
fastbook.setup_book()

Mounted at /content/gdrive


In [4]:
from fastai.text.all import *

In [6]:
Path.cwd()

Path('/content')

In [30]:
data_path = fastbook.gdrive / Path('data')
data_file = data_path / 'mtsamples.csv'

In [31]:
data_file

Path('/content/gdrive/My Drive/data/mtsamples.csv')

In [32]:
if data_file.is_file():
    print('Already downloaded')
else:
    print('downloading data file')
    download_data(
        url='https://github.com/socd06/medical-nlp/raw/master/data/mtsamples.csv',
        fname = data_file
    )

downloading data file


In [10]:
mtsamples = pd.read_csv(str(data_path) + '/mtsamples.csv')
mtsamples.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with complaint of allergies.,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it...","allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,"
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climbing stairs, difficulty with airline seats, tying shoes, used to public seating, and lifting objects off the floor. He exercises three times a week at home and does cardio. He has difficulty walking two blocks or five flights of stairs. Difficulty with snoring. He has muscle and joint pains including knee pain, back pain, foot and ankle pain, and swelling. He has gastroesophageal reflux disease.,PAST SURGICAL HISTORY:, Includes reconstructive surgery on his right hand 13 years ago. ,SOCIAL HISTORY:, He is currently single. He has about te...","bariatrics, laparoscopic gastric bypass, weight loss programs, gastric bypass, atkin's diet, weight watcher's, body weight, laparoscopic gastric, weight loss, pounds, months, weight, laparoscopic, band, loss, diets, overweight, lost"
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC today. He is a very pleasant gentleman who is 42 years old, 344 pounds. He is 5'9"". He has a BMI of 51. He has been overweight for ten years since the age of 33, at his highest he was 358 pounds, at his lowest 260. He is pursuing surgical attempts of weight loss to feel good, get healthy, and begin to exercise again. He wants to be able to exercise and play volleyball. Physically, he is sluggish. He gets tired quickly. He does not go out often. When he loses weight he always regains it and he gains back more than he lost. His biggest...","bariatrics, laparoscopic gastric bypass, heart attacks, body weight, pulmonary embolism, potential complications, sleep study, weight loss, gastric bypass, anastomosis, loss, sleep, laparoscopic, gastric, bypass, heart, pounds, weight,"
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement with left atrial diameter of 4.7 cm.,2. Normal size right and left ventricle.,3. Normal LV systolic function with left ventricular ejection fraction of 51%.,4. Normal LV diastolic function.,5. No pericardial effusion.,6. Normal morphology of aortic valve, mitral valve, tricuspid valve, and pulmonary valve.,7. PA systolic pressure is 36 mmHg.,DOPPLER: , ,1. Mild mitral and tricuspid regurgitation.,2. Trace aortic and pulmonary regurgitation.","cardiovascular / pulmonary, 2-d m-mode, doppler, aortic valve, atrial enlargement, diastolic function, ejection fraction, mitral, mitral valve, pericardial effusion, pulmonary valve, regurgitation, systolic function, tricuspid, tricuspid valve, normal lv"
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,"1. The left ventricular cavity size and wall thickness appear normal. The wall motion and left ventricular systolic function appears hyperdynamic with estimated ejection fraction of 70% to 75%. There is near-cavity obliteration seen. There also appears to be increased left ventricular outflow tract gradient at the mid cavity level consiste

In [11]:
mtsamples.transcription.str.len()

0       1331.0
1       2431.0
2       4422.0
3        495.0
4       1618.0
         ...  
4994    5338.0
4995    1566.0
4996    4543.0
4997    2781.0
4998    4379.0
Name: transcription, Length: 4999, dtype: float64

In [12]:
mtsamples.medical_specialty.value_counts()

 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               

This is one way to load the data for training

```python
TextDataLoaders.from_df(mtsamples,
                        valid_pct=0.2,
                        seed=42,
                        text_col='transcription',
                        label_col='medical_specialty',
                        seq_len=72)
```

In [13]:
# dls_lm = DataBlock(
#     blocks=(TextBlock.from_df('transcription', is_lm=True), CategoryBlock),
#     get_x=ColReader('transcription'), 
#     get_y=ColReader('medical_specialty'),
#     splitter=TrainTestSplitter(test_size=0.2,
#                                random_state=42,
#                                stratify=mtsamples.medical_specialty)
# )

In [14]:
dls_lm = DataBlock(
    blocks=TextBlock.from_df('transcription', is_lm=True),
    get_x=ColReader('text'),
    splitter=TrainTestSplitter(test_size=0.2,
                               random_state=42)
)

In [15]:
# setting max transcription length to 128 words
# this step takes 1 - 2 minutes
dls_lm = dls_lm.dataloaders(mtsamples, bs=64, seq_len=128)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [16]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxup procedure : , xxmaj flexible bronchoscopy . ,preoperative xxup diagnosis ( es ) : , xxmaj chronic wheezing . ,indications xxup for xxup procedure : , xxmaj evaluate the airway . ,description xxup of xxup procedure : , this was done in the pediatric endoscopy suite with the aid of xxmaj anesthesia . xxmaj the patient was sedated with xxunk and propofol . xxmaj one ml of 1 % lidocaine was used for airway anesthesia . xxmaj the 2.8-mm flexible pediatric bronchoscope was passed through the left naris . xxmaj the upper airway was visualized . xxmaj the epiglottis , xxunk , and vocal cords were all normal . xxmaj the scope was passed below the cords . xxmaj the subglottic space was normal . xxmaj","xxup procedure : , xxmaj flexible bronchoscopy . ,preoperative xxup diagnosis ( es ) : , xxmaj chronic wheezing . ,indications xxup for xxup procedure : , xxmaj evaluate the airway . ,description xxup of xxup procedure : , this was done in the pediatric endoscopy suite with the aid of xxmaj anesthesia . xxmaj the patient was sedated with xxunk and propofol . xxmaj one ml of 1 % lidocaine was used for airway anesthesia . xxmaj the 2.8-mm flexible pediatric bronchoscope was passed through the left naris . xxmaj the upper airway was visualized . xxmaj the epiglottis , xxunk , and vocal cords were all normal . xxmaj the scope was passed below the cords . xxmaj the subglottic space was normal . xxmaj the"
1,"was then inspected for any remaining short bony edges , none were noted . ,copious amounts of sterile saline was then used to flush the surgical site . xxmaj the capsule was closed with # 3 - 0 xxmaj vicryl . xxmaj subcutaneous closure was performed with # 4 - 0 xxmaj vicryl followed by running subcuticular # 5 - 0 xxmaj vicryl . steri - strips were applied and 1 cc of dexamethasone phosphate was injected into the surgical site . ,dressings consisted of # 0 - 1 silk , copious xxmaj betadine , 4 x 4s , xxmaj kling , xxmaj kerlix , and xxmaj coban . xxmaj the pneumatic ankle tourniquet was released and immediate hyperemic flush was noted to all five digits of the","then inspected for any remaining short bony edges , none were noted . ,copious amounts of sterile saline was then used to flush the surgical site . xxmaj the capsule was closed with # 3 - 0 xxmaj vicryl . xxmaj subcutaneous closure was performed with # 4 - 0 xxmaj vicryl followed by running subcuticular # 5 - 0 xxmaj vicryl . steri - strips were applied and 1 cc of dexamethasone phosphate was injected into the surgical site . ,dressings consisted of # 0 - 1 silk , copious xxmaj betadine , 4 x 4s , xxmaj kling , xxmaj kerlix , and xxmaj coban . xxmaj the pneumatic ankle tourniquet was released and immediate hyperemic flush was noted to all five digits of the right"


In [34]:
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    model_dir=fastbook.gdrive / Path('data'),
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [35]:
# on macbook this takes 40 minues
# on GPU machine this takes about 1 minute
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.048979,2.881843,0.449544,17.847141,01:16


In [36]:
learn.save('1epoch')

Path('/content/gdrive/My Drive/data/1epoch.pth')

In [37]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.661049,2.703400,0.470938,14.930404,01:23
1,2.439440,2.477741,0.501964,11.914323,01:23
2,2.142361,2.210938,0.542657,9.124273,01:23
3,1.807399,1.970857,0.586664,7.176824,01:23
4,1.549410,1.749228,0.631957,5.750159,01:23
5,1.329410,1.576361,0.669718,4.837323,01:23
6,1.091925,1.441991,0.702111,4.229109,01:23
7,0.960812,1.352861,0.724349,3.868476,01:23
8,0.876318,1.315484,0.733831,3.726555,01:24
9,0.805579,1.309688,0.735736,3.705017,01:24


In [ ]:
learn.save_encoder('finetuned')

In [ ]:
dls_lm = DataBlock(
    blocks=TextBlock.from_df('transcription', is_lm=True),
    get_x=ColReader('text'),
    #get_y='medical_specialty',
    splitter=TrainTestSplitter(test_size=0.2,
                               random_state=42,
                               stratify=mtsamples.medical_specialty)
)